In [1]:
import sys
sys.path.append("../../src")
import os
import datetime
import pandas as pd
import numpy as np
from example_lorenz import get_lorenz_data
from sindy_utils import library_size
from training import train_network
import tensorflow as tf

tf.compat.v1.disable_eager_execution()
tf.compat.v1.disable_resource_variables()

Instructions for updating:
non-resource variables are not supported in the long term


# Generate data

In [2]:
# generate training, validation, testing data
noise_strength = 1e-6
training_data = get_lorenz_data(1024, noise_strength=noise_strength)
validation_data = get_lorenz_data(20, noise_strength=noise_strength)

# Set up model and training parameters

In [3]:
params = {}

params['input_dim'] = 128
params['latent_dim'] = 3
params['model_order'] = 1
params['poly_order'] = 3
params['include_sine'] = False
params['library_dim'] = library_size(params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_z'] = 0.0
params['loss_weight_sindy_x'] = 1e-4
params['loss_weight_sindy_regularization'] = 1e-5

params['activation'] = 'sigmoid'
params['widths'] = [64,32]

# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 1024
params['learning_rate'] = 1e-3

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100

# training time cutoffs
params['max_epochs'] = 5001
params['refinement_epochs'] = 1001

# Run training experiments

In [4]:
num_experiments = 1
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'lorenz_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")

    tf.compat.v1.reset_default_graph()

    results_dict = train_network(training_data, validation_data, params)
    df = df.append({**results_dict, **params}, ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0
TRAINING


2024-05-23 10:33:40.472401: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2024-05-23 10:33:40.498953: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 0
   training loss 0.03483261913061142, (0.03402067, 181.20525, 8.034021, 0.85467696)
   validation loss 0.031022926792502403, (0.030033039, 192.3315, 9.813403, 0.85467696)
decoder loss ratio: 0.165900, decoder SINDy loss  ratio: 1.000711
Epoch 100
   training loss 6.186501559568569e-05, (1.764493e-05, 0.51262766, 0.23224023, 2.0996063)
   validation loss 6.44523388473317e-05, (1.6538967e-05, 0.5295298, 0.26917312, 2.0996063)
decoder loss ratio: 0.000091, decoder SINDy loss  ratio: 0.027449
Epoch 200
   training loss 4.2115148971788585e-05, (1.1445893e-05, 0.12365005, 0.07056171, 2.3613083)
   validation loss 4.194035136606544e-05, (1.0819112e-05, 0.1308358, 0.075081564, 2.3613083)
decoder loss ratio: 0.000060, decoder SINDy loss  ratio: 0.007656
Epoch 300
   training loss 4.530609294306487e-05, (1.925756e-05, 0.097754955, 0.044370096, 2.1611521)
   validation loss 4.4727094064000994e-05, (1.9017529e-05, 0.07841156, 0.040980447, 2.1611521)
decoder loss ratio: 0.000105, decoder SI